# Yahoo 電影爬蟲練習：爬取電影放映查詢結果
1. 獲取電影的代號、放映地區、放映日期
2. 再送出查詢給伺服器

Hint: 
- 片名列表 API 位在 `<select>` 標籤內的 `data-url` 
  - Ans: https://movies.yahoo.com.tw/ajax/in_theater_movies


In [ ]:
from bs4 import BeautifulSoup
import re
import requests

### 先搜尋全部的電影代號(ID)資訊

In [ ]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = "https://movies.yahoo.com.tw/ajax/in_theater_movies"
movie_ids = requests.get(url).json()
movie_ids

{'10159': '捍衛戰士：獨行俠',
 '10462': '魔比斯',
 '10897': '尼羅河謀殺案',
 '10904': '蝙蝠俠',
 '11373': '一家之主',
 '11477': '咒',
 '11545': '素還真',
 '11588': '秘境探險',
 '11626': '劫命救護',
 '11755': '娶我吧！',
 '11809': '侏羅紀世界：統霸天下',
 '11939': '天堂計劃',
 '11999': '音速小子2',
 '12030': '怪獸與鄧不利多的秘密',
 '12033': '壞蛋聯盟',
 '12136': '奇異博士2：失控多重宇宙',
 '12274': '在車上',
 '12284': '反恐行動：獨立日',
 '12385': '劇場版 咒術迴戰 0',
 '12405': '世界上最爛的人',
 '12430': '頭七',
 '12434': '那些得不到保護的人',
 '12444': '黑光行動',
 '12606': '獵殺戰場',
 '12623': '丟包大作戰',
 '12647': '貝爾法斯特',
 '12648': '全面掃蕩',
 '12667': '售命',
 '12687': '少年吔',
 '12791': '電影版 如果30歲還是處男，似乎就能成為魔法師',
 '12799': '露草',
 '12800': '給阿媽的一封信',
 '12804': '媽的多重宇宙',
 '12828': '友你才精彩',
 '12886': '急急復雞雞',
 '12908': '重慶森林',
 '12921': '輪到你了 劇場版',
 '12923': '犯罪都市2',
 '12924': '牛首村',
 '12925': '電影版 99.9 不可能的翻案',
 '12973': '漁港的肉子',
 '12976': '藍色是最溫暖的顏色',
 '12977': '終極夜路',
 '12980': '進擊的地才',
 '12981': '我的夏日大戀習',
 '13035': '教練',
 '13077': '鯊顫',
 '13108': '費爾的旅程',
 '13152': '空氣殺人',
 '13162': '猛鬼大學：第2學期',
 '13167': '愛在暹

### 指定你有興趣的電影其ID，並查詢其放映地區資訊
- Hint: API URL 會在你選好片名時，瀏覽器傳送請求，出現在 Network Panel 列表，可以觀察一下

In [ ]:
# 參考前一個步驟中擷取到的 ID 資訊，並指定 ID
movie_id = 10159

In [ ]:
url = "https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater"
payload = {'movie_id': str(movie_id)}

# headers 中要有 mv-authorization 才能得到回傳結果，你可以試著把 mv-authorization 拿掉試試看會回傳什麼訊息
# 其他的資訊可有可無，不過為了讓學員學好學滿，還是附上完整的 headers 資訊會長怎樣
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
res = requests.get(url, params=payload, headers=headers).json()
print(res)
# 整理放映地區資料
publish_area = {item["title"]: item["area_id"] for item in res}
print(publish_area)


[{'area_id': 28, 'title': '台北市', 'priority': 150, 'ts_update': 1547519309}, {'area_id': 8, 'title': '新北市', 'priority': 145, 'ts_update': 1547194527}, {'area_id': 16, 'title': '桃園', 'priority': 140, 'ts_update': 1547452573}, {'area_id': 20, 'title': '新竹', 'priority': 135, 'ts_update': 1547452603}, {'area_id': 15, 'title': '苗栗', 'priority': 130, 'ts_update': 1547452620}, {'area_id': 2, 'title': '台中', 'priority': 125, 'ts_update': 1547507368}, {'area_id': 10, 'title': '台南', 'priority': 100, 'ts_update': 1547507403}, {'area_id': 17, 'title': '高雄', 'priority': 95, 'ts_update': 1547507403}]
{'台北市': 28, '新北市': 8, '桃園': 16, '新竹': 20, '苗栗': 15, '台中': 2, '台南': 10, '高雄': 17}


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期
- Hint: API URL 會在你按下查詢鍵時，出現在 Network Panel 列表，可以觀察一下


In [ ]:
# 指定放映地區
area_id = publish_area["台北市"]

In [ ]:
# 向網站發送請求
url = "https://movies.yahoo.com.tw/movietime_result.html"
payload = {"movie_id": str(movie_id), "area_id": str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = "utf-8"

soup = BeautifulSoup(resp.text, 'html.parser')
movie_dates = soup.find_all("label", attrs={'for': re.compile("date_[\d]")})  # 利用 regex 篩選符合條件的 <label></label> 節點
movie_dates = [date.p.string + " " + date.h3.string for date in movie_dates]
print(movie_dates)


['六月 11', '六月 12', '六月 13', '六月 14', '六月 15']


### 指定觀看日期，查詢放映資訊
- 印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊
> - Hint: 你在頁面上選擇日期時，瀏覽器傳送請求，此時 API 會出現在 Network Panel 列表，可以觀察一下


In [ ]:
# 選定要觀看的日期
date = "2022-06-11"

In [ ]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {
    "movie_id": str(movie_id),
    "date": date,
    "area_id": str(area_id),
    "theater_id": "",
    "datetime": "",
    "movie_type_id": ""
}

res = requests.get(url, params=payload).json()
soup = BeautifulSoup(res["view"], "html.parser")
html = soup.find_all("ul", attrs={'data-theater_name': re.compile(".*")})
print(html)

[<ul class="area_time _c jq_area_time" data-theater_name="台北美麗華大直影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=32" data-theater_url="http://www.miramarcinemas.tw/" id="theater_id_32">
<li class="adds">
<a href="https://movies.yahoo.com.tw/theater_result.html/id=32">台北美麗華大直影城</a>
<span>02-85022208</span>
</li>
<li class="taps">
<span class="tapR">數位</span> </li>
<li class="time _c">
<div class="input_picker jq_input_picker">
<input class="gabtn" data-cinema="40288eb0050f31210ada0f55676b1bbc" data-date="1654876800" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '捍衛戰士：獨行俠']" data-movie="337661e23dca4cf2b20c9aa8b12b206a" data-movie_date="06.11" data-movie_time="09:10" data-movie_title="捍衛戰士：獨行俠" data-movie_type="數位" data-session="7bef7c772fa149239f00c9356e26109b" data-time="1654909800" id="78574915" name="schedule_list" type="radio" value="2022-06-11 09:10:00"/>
<label class="select" for="78574915">09:10</label>
<input class="gabtn" data-cinema="40

In [ ]:
# 列印上映電影院以及時間
for movie_theater in html:
    print("----------------------------------------------------------------------")
    
    theater_name = movie_theater.find("li", attrs={'class':'adds'})
    print("電影院: ", theater_name.a.string)

    project_type = movie_theater.find_all("li", attrs={'class':'taps'})  # 定位放映類型
        
    time_table = movie_theater.find_all("li", attrs={'class':'time _c'})  # 定位時間表

    for i in range(len(project_type)):
        print("放映類型: ", project_type[i].span.get_text())  # 印出電影類型
        
        for p4 in time_table[i].find_all("input", attrs={'name':'schedule_list'}):
            print("%s" % (p4["value"]))  # 印出放映時間


----------------------------------------------------------------------
電影院:  台北美麗華大直影城
放映類型:  數位
2022-06-11 09:10:00
2022-06-11 11:05:00
2022-06-11 11:50:00
2022-06-11 12:40:00
2022-06-11 13:40:00
2022-06-11 14:30:00
2022-06-11 15:20:00
2022-06-11 16:20:00
2022-06-11 17:05:00
2022-06-11 18:00:00
2022-06-11 19:15:00
2022-06-11 19:40:00
2022-06-11 20:40:00
2022-06-11 21:20:00
2022-06-11 22:20:00
2022-06-11 23:20:00
2022-06-12 00:00:00
----------------------------------------------------------------------
電影院:  台北新光影城
放映類型:  數位
2022-06-11 10:15:00
2022-06-11 15:15:00
2022-06-11 17:45:00
2022-06-11 20:15:00
----------------------------------------------------------------------
電影院:  台北in89豪華數位影城
放映類型:  數位
2022-06-11 11:10:00
2022-06-11 13:40:00
2022-06-11 16:10:00
2022-06-11 18:40:00
2022-06-11 21:10:00
2022-06-11 23:40:00
----------------------------------------------------------------------
電影院:  喜滿客絕色影城
放映類型:  數位
2022-06-11 10:30:00
2022-06-11 11:30:00
2022-06-11 13:00:00
2022-06-11 14: